In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
Path_to_data = "../data/AdSmartABdata.csv"
#Read data from csv file
data = pd.read_csv(Path_to_data, na_values=['?', None])

#Display sample of the data
data.sample(10)

,auction_id,experiment,date,hour,device_make,platform_os,browser,yes,no
4799,98f3af12-cd6e-423d-b86f-1afa0dfd0b30,control,2020-07-08,18,Samsung $2,6,Samsung Internet,0,0
7981,fc721b18-cd69-4bdf-a177-64d6af3f4e8e,control,2020-07-08,17,Generic Smartphone,6,Chrome Mobile,0,0
1346,2c45d006-971c-4a7e-b1e4-f96addeee0cf,control,2020-07-03,15,Generic Smartphone,6,Chrome Mobile,0,0
1782,3b427db1-4b52-446f-bcb2-2f6039ad1d42,exposed,2020-07-03,9,Generic Smartphone,6,Chrome Mobile,1,0
5029,a0398e7c-61fc-41c2-bda2-1891b8365e20,exposed,2020-07-05,7,Generic Smartphone,6,Chrome Mobile,0,0
4332,8a89116b-a35f-40d0-b9c2-8fd9351df037,exposed,2020-07-04,0,Samsung SM-G920F,6,Chrome Mobile WebView,0,0
4387,8c07c329-9a3a-4051-8cca-71a0d1af5421,control,2020-07-03,15,Samsung SM-G770F,6,Facebook,0,0
6070,c0cd6b96-dfd3-4ccc-bbeb-435330a05dc2,exposed,2020-07-03,12,OnePlus ONEPLUS A6013,6,Chrome Mobile WebView,1,0
2989,6060009d-20fe-4ecf-bc9f-0a3e1cb5e815,exposed,2020-07-03,11,Generic Smartphone,6,Chrome Mobile,0,1
5967,bd4a46d1-b536-4a86-9234-20d926a333cf,control,2020-07-08,15,Samsung SM-G973F,6,Samsung Internet,0,0


In [4]:
def transform_data(df):
  '''
  segment data into exposed and control groups
  consider that SmartAd runs the experment hourly, group data into hours. 
      Hint: create new column to hold date+hour and use df.column.map(lambda x:  pd.Timestamp(x,tz=None).strftime('%Y-%m-%d:%H'))
  create two dataframes with bernouli series 1 for posetive(yes) and 0 for negative(no)
    Hint: Given engagement(sum of yes and no until current observation as an array) and success (yes countas an array), the method generates random binomial distribution
        #Example
           engagement = np.array([5, 3, 3])
           yes = np.array([2, 0, 3])       
         Output is "[1] 1 0 1 0 0 0 0 0 1 1 1", showing a binary array of 5+3+3 values
         of which 2 of the first 5 are ones, 0 of the next 3 are ones, and all 3 of
         the last 3 are ones where position the ones is randomly distributed within each group.
  '''
  return data_stream

In [5]:
def transform_data(df, sample_size):
    '''
    segment data into exposed and control groups
    consider that SmartAd runs the experment hourly, group data into hours. 
      Hint: create new column to hold date+hour and use df.column.map(lambda x:  pd.Timestamp(x,tz=None).strftime('%Y-%m-%d:%H'))
    create two dataframes with bernouli series 1 for posetive(yes) and 0 for negative(no)
    Hint: Given engagement(sum of yes and no until current observation as an array) and success (yes countas an array), 
    the method generates random binomial distribution
        #Example
           engagement = np.array([5, 3, 3])
           yes = np.array([2, 0, 3])       
         Output is "[1] 1 0 1 0 0 0 0 0 1 1 1", showing a binary array of 5+3+3 values
         of which 2 of the first 5 are ones, 0 of the next 3 are ones, and all 3 of
         the last 3 are ones where position the ones is randomly distributed within each group.
    '''
    exposed = df[df['clicked'] == 1][df['experiment'] == 'exposed']
    control = df[df['clicked'] == 1][df['experiment'] == 'control']
    exposed = np.array(exposed.sample(sample_size)['yes'])
    control = np.array(control.sample(sample_size)['yes'])
    return exposed,control

def plotDataSummary(exposed, control):
    fig, axs = plt.subplots(1,2,figsize=(16,4))
    fig.suptitle('Yes-no distribution per group')
    sns.countplot(x = 'yes', data=pd.DataFrame({'yes' : exposed}),ax=axs[0]).set_title('Exposed Group')
    sns.countplot(x = 'yes', data=pd.DataFrame({'yes' : control}),ax=axs[1]).set_title('Control Group')
    
def pretyPrintTestResult(test):
    '''This function print final test result. Json format is recommended. For example'''
    message = test.perform_test()
    res = {
    "name": "Sequential AB Testing",
    "engagementCountControl": len(test.control),
    "engagementCountExposed": len(test.exposed),
    "positiveCountControl": test.C,
    "positiveCountExposed": test.T,
    "TotalObservation": message
    }
    return res

In [ ]:
class ConditionalSPRT:
  def __init__(self,exposed,control,odd_ratio,alpha=0.05,beta=0.10,stop=None):
    self.exposed = exposed
    self.control = control
    self.odd_ratio = odd_ratio
    self.alpha = alpha
    self.beta = beta
    self.stop = stop

In [ ]:
def ConditionalSPRT(self,x,y,t1,alpha=0.05,beta=0.10,stop=None):
  """
   #
  # Meeker's SPRT for matched `x` (treatment) and `y` (control), 
  # both indicator responses, likelihood ratio t1, error rates alpha and beta,
  # and (optionally) truncation after trial stop.
  #
  # The return variable contains these elements:
  #(outcome,n, k,l,u,truncated,truncate_decision,x1,r,stats,limits)
  # * outcome:   "continue," "reject null," or "accept null".
  # * n: number observation used for the decsion
  # * k:     Index at which the outcome decision was made (or NA)
  # * l:     lower critical point
  # * u:     upper critical point
  # * truncate_decision: The approximate decision made after truncate point
  # * truncated: If the test was truncated, the value of `n.0`; NA otherwise
  # * x1:       Original data `x`, cumulative
  # * r:         Cumulative sum of x+y
  # * stats:     Series of cumulative sums of log probability ratios
  # * limits:    Two rows giving lower and upper critical limits, respectively
  #
  """
        if t1<=1:
            printLog('warning',"Odd ratio should exceed 1.")
        if (alpha >0.5) | (beta >0.5):
            printLog('warning',"Unrealistic values of alpha or beta were passed."
                     +" You should have good reason to use large alpha & beta values")
        if stop!=None:
            stop=math.floor(n0)

        def comb(n, k):
            return factorial(n) // factorial(k) // factorial(n - k)
        
        def lchoose(b, j):
            a=[]
            if (type(j) is list) | (isinstance(j,np.ndarray)==True):
                if len(j)<2:
                    j=j[0]
            if (type(j) is list) | (isinstance(j,np.ndarray)==True):
                for k in j:
                    n=b
                    if (0 <= k) & (k<= n):
                        a.append(math.log(comb(n,k)))
                    else:
                        a.append(0)
            else:
                n=b
                k=j
                if (0 <= k) & (k<= n):
                    a.append(math.log(comb(n,k)))
                else:
                    a.append(0)

            return np.array(a)

        def g(x,r,n,t1,t0=1):
          """
         #
          # Meeker's (1981) function `g`, the log probability ratio.
          # 
          """
            return -math.log(h(x,r,n,t1))+math.log(h(x,r,n,t0))

        def h(x,r,n,t=1):
          """
          #
          # Reciprocal of Meeker's (1981) function `h`: the conditional probability of 
          # `x` given `r` and `n`, when the odds ratio is `t`.
          #
          # `x` is his "x1", the number of positives in `n` control trials.
          # `r` is the total number of positives.
          # `n` is the number of (control, treatment) pairs.
          # `t` is the odds ratio.
          #
          """
            return f(r,n,t,offset=ftermlog(x,r,n,t))

        def f(r,n,t,offset=0):
          """#
          # Meeker's (1981) function exp(F(r,n,t)), proportional to the probability of 
          #  `r` (=x1+x2) in `n` paired trials with an odds ratio of `t`.
          #
          # This function does *not* vectorize over its arguments.
          #"""
            upper=max(0,r-n)
            lower=min(n,r)
            rng=list(range(upper,lower+1))
            return np.sum(fterm(rng,r,n,t,offset))

        def fterm(j,r,n,t,offset=0):
            ftlog=ftermlog(j,r,n,t,offset)
            return np.array([math.exp(ex) for ex in ftlog])

        def ftermlog(j,r,n,t,offset=0):
          """
          #
          # Up to an additive constant, the log probability that (x1, x1+x2) = (j, r) 
          # in `n` paired trials with odds ratio of `t`.
          #
          # `offset` is used to adjust the result to avoid under/overflow.
          #
          """
            xx=r-j
            lch=lchoose(n,j)
            lchdiff=lchoose(n,xx)
            lg=np.array(j)*math.log(t)
            lgsum=lch+lchdiff
            lgsum2=lgsum+lg
            lgdiff=lgsum2-offset

            return lgdiff

        def logf(r,n,t,offset=0):
          """
          #
          # A protected vesion of log(f), Meeker's function `F`.
          #
          """
            z=f(r,n,t,offset)
            if z>0:
                return math.log(z)
            else:
                return np.nan

        def clowerUpper(r,n,t1c,t0=1,alpha=0.05,beta=0.10):
          """
           #
          # Meeker's (1981) functions c_L(r,n) and c_U(r,n), the  critical values for x1.
          # 0 <= r <= 2n; t1 >= t0 > 0.
          #
          """
            offset=ftermlog(math.ceil(r/2),r,n,t1c)
            z=logf(r,n,t1c,logf(r,n,t0,offset)+offset)
            a=-math.log(alpha/(1-beta))
            b=math.log(beta/(1-alpha))
            lower=b
            upper=1+a
            return (np.array([lower,upper])+z)/math.log(t1c/t0)

        l=math.log(beta/(1-alpha))
        u=-math.log(alpha/(1-beta))
        sample_size=min(len(x),len(y))
        n=np.array(range(1,sample_size+1))

        if stop!=None:
            n=np.array([z for z in n if z<=stop])
        x1=np.cumsum(x[n-1])
        r=x1+np.cumsum(y[n-1])
        stats=np.array(list(map(g,x1, r, n, [t1]*len(x1)))) #recurcively calls g
         #
          # Perform the test by finding the first index, if any, at which `stats`
          # falls outside the open interval (l, u).
          #
        clu=list(map(clowerUpper,r,n,[t1]*len(r),[1]*len(r),[alpha]*len(r), [beta]*len(r)))
        limits=[]
        for v in clu:
            inArray=[]
            for vin in v:
                inArray.append(math.floor(vin))
            limits.append(np.array(inArray))
        limits=np.array(limits)

        k=np.where((stats>=u) | (stats<=l))
        cvalues=stats[k]
        if cvalues.shape[0]<1:
            k= np.nan
            outcome='Unable to conclude.Needs more sample.'
        else:
            k=np.min(k)
            if stats[k]>=u:
                outcome=f'Exposed group produced a statistically significant increase.'
            else:
                outcome='Their is no statistically significant difference between two test groups'
        if (stop!=None) & (k==np.nan):
          #
          # Truncate at trial stop, using Meeker's H0-conservative formula (2.2).
          # Leave k=NA to indicate the decision was made due to truncation.
          #
            c1=clowerUpper(r,stop,t1,alpha,beta)
            c1=math.floor(np.mean(c1)-0.5)
            if x1[n0]<=c1:
                truncate_decision='h0'
                outcome='Maximum Limit Decision. The aproximate decision point shows their is no statistically significant difference between two test groups'
            else:
                truncate_decision='h1'
                outcome=f'Maximum Limit Decision. The aproximate decision point shows exposed group produced a statistically significant increase.'
            truncated=stop
        else:
            truncate_decision='Non'
            truncated=np.nan
        return (outcome,n, k,l,u,truncated,truncate_decision,x1,r,stats,limits)